## 1. Instalación del navegador

Necesitamos utilizar un navegador, para ellos se implementa el siguiente código que nos va a permitir utilizar Chrome como navegador desde Google Colab.

Fuente: https://www.youtube.com/watch?v=MYD7hnxjiRw

In [1]:
# Instalar dependencias para utilizar Google Chrome
!sudo apt-get update
!sudo apt-get install -y libu2f-udev
!pip install selenium

# Instalar el navegador
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f
!rm -rf google-chrome-stable_current_amd64.deb

# Descargar el WebDriver 
!wget https://chromedriver.storage.googleapis.com/112.0.5615.49/chromedriver_linux64.zip
!unzip chromedriver_linux64
!rm -rf chromedriver_linux64.zip
!rm -rf LICENSE.chromedriver

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [973 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,59

## 2. Cargamos las librerias que vamos a utilizar

In [2]:
# navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# manejo de lista desplegable
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

# manejo de tiempos
import time

# manejo de datos
import pandas as pd

## 3. Opciones del navegador y obtención de la URL objetivo

In [3]:
# Ruta de la página web
url = 'https://baloncestoenvivo.feb.es/rankings/lfendesa/4/2022'

# Opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') # deshabilita el modo sandbox
chrome_options.add_argument('--start_maximized')
chrome_options.add_argument('--disable-web-security')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--allow-running-insecure-content')
chrome_options.add_argument('--no-default-browser-check')
chrome_options.add_argument('--no-first-run')
chrome_options.add_argument('--no-proxy-server')

# Ruta del controlador de Chrome
chrome_driver_path = '/content/chromedriver'

# Crear un objeto Service para especificar la ruta del controlador
service = Service(chrome_driver_path)

# Crear una instancia del navegador Chrome
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navegar a una página web
driver.get(url)

## 4. Manejar lista desplegable

Fuente: https://www.youtube.com/watch?v=CHiwaFEUB1Y (min 16)

Tenemos una lista desplegable en la que podemos seleccionar con id únicos: 

*   **temporada:** _ctl0_MainContentPlaceHolderMaster_temporadasDropDownList
*   **competición:** _ctl0_MainContentPlaceHolderMaster_gruposDropDownList
*   **atributos:** _ctl0_MainContentPlaceHolderMaster_rankingsDropDownList
*   **nacionalidad:** _ctl0_MainContentPlaceHolderMaster_nacionalDropDownList

El resto de selectores podemos dejarlos como están ya que nos interesa recopilar los datos acumulados de la temporada.











In [4]:
# seleccionamos las opciones del menú desplegable
# habrá que hacer una función

dropdownT = Select(driver.find_element('id', '_ctl0_MainContentPlaceHolderMaster_temporadasDropDownList'))
dropdownT.select_by_visible_text('2021/2022')
dropdownC = Select(driver.find_element('id', '_ctl0_MainContentPlaceHolderMaster_gruposDropDownList'))
dropdownC.select_by_visible_text('Liga Regular Único')
dropdownA = Select(driver.find_element('id', '_ctl0_MainContentPlaceHolderMaster_rankingsDropDownList'))
dropdownA.select_by_visible_text('Puntos')
#dropdownN = Select(driver.find_element('id', '_ctl0_MainContentPlaceHolderMaster_nacionalDropDownList'))
#dropdownN.select_by_visible_text('Nacional')

In [5]:
# obtenemos el número de páginas del paginador
total_pages = len(driver.find_element(By.CLASS_NAME,'tabla-paginador').find_elements(By.TAG_NAME,'a'))
total_pages

5

Para hacer scroll down
Fuente: https://www.youtube.com/watch?v=Z32K-8uy2mg

In [6]:
# futura función que recopila los datos de los atributos
lista = []
page = 2
while page <= total_pages + 2:
  time.sleep(2)  
  selection = driver.find_element(By.ID, '_ctl0_MainContentPlaceHolderMaster_rankingAcumuladosDataGrid').find_elements(By.TAG_NAME,'td')
  for sel in selection:
    if sel.text != '' and sel.text != '1 2 3 4 5 6':
      lista.append(sel.text)
  # Para hacer scroll
  driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
  # Damos tiempo
  time.sleep(2)
  # Hacemos clic en el siguiente enlace de paginación
  if page < total_pages + 2:
    button_page = driver.find_element(By.LINK_TEXT, str(page)).click()
  page = page + 1

In [ ]:
# obtenemos un dataframe con los puntos
df_puntos = pd.DataFrame(columns=['jugadora', 'equipo', 'puntos totales', 'partidos', 'media puntos totales'])

for i in range(int(len(lista)/5)):
  df_puntos = df_puntos.append({'jugadora': lista[5*i], 'equipo': lista[5*i+1], 'puntos totales':lista[5*i+2], 'partidos':lista[5*i+3], 'media puntos totales':lista[5*i+4]}, ignore_index=True)

In [8]:
df_puntos

,jugadora,equipo,puntos totales,partidos,media puntos totales
0,"ATKINSON, TANAYA MECHELLE",KUTXABANK ARASKI,459,28,"16,39"
1,"CALDWELL, MAYA DENISE",SPAR GRAN CANARIA,404,26,"15,54"
2,"PLAISANCE, THERESA MONIQUE",TENERIFE,245,16,"15,31"
3,"COULIBALY, MARIAM ALOU",IDK EUSKOTREN,412,27,"15,26"
4,"BURKE, KENNEDY VALENTINE",SPAR GIRONA,408,27,"15,11"
...,...,...,...,...,...
149,"CONDE DURAN, ITSASO",MOVISTAR ESTUDIANTES,21,18,"1,17"
150,"ESNAL AGUIRRE, ANE",IDK EUSKOTREN,13,15,"0,87"
151,"HERNANDEZ ORTIZ, ZOE",CASADEMONT ZARAGOZA,18,25,"0,72"
152,"ALBERDI LARRUCEA, MARTA",LOINTEK GERNIKA BIZKAIA,12,20,"0,6"


In [ ]:
# Cerrar el navegador
#driver.quit()